In [1]:
import pandas as pd
import awswrangler as awr
import openpyxl


In [2]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_baixadas.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_pagamentos = awr.athena.read_sql_query(query, database='silver')


In [3]:
df_pagamentos = df_pagamentos.drop_duplicates('ponteiro', keep='first')
df_pagamentos = df_pagamentos[df_pagamentos['data_baixa']>= pd.to_datetime('2025-01-31').date()]



In [4]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_inadimplentes.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_inadimplentes = awr.athena.read_sql_query(query, database='silver')
df_inadimplentes = df_inadimplentes.drop_duplicates('ponteiro', keep='first')

estabelecendo a base de vencidos

In [5]:
today = pd.to_datetime('today').date()
month = today.month
df_inadimplentes['data_vencimento'] = pd.to_datetime(df_inadimplentes['data_vencimento'])


base_vencidos = df_inadimplentes[df_inadimplentes['data_vencimento']>= pd.to_datetime('2025-01-31')]
#base_vencidos = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.month == month]


In [6]:
base_vencidos['situacao'] = pd.NA
base_vencidos['data_atualizacao'] = pd.NA
base_vencidos.sort_values(by='data_vencimento', ascending=False)
base_vencidos.head()


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_22224\1941425127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos['situacao'] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_22224\1941425127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos['data_atualizacao'] = pd.NA


,codigo_cadastro,associado,grupo,matricula,conjunto,aplicacao_financeira,data_emissao,data_vencimento,valor_titulo,codigo,...,numero_boleto,nosso_numero,status,id_set,parent,cooperativa,placa,chassi,situacao,data_atualizacao
0,4891,VOLMAR DE ALMEIDA,RECEITAS,20400,10238,Mensalidades/Benefícios,2024-08-05,2025-06-05,1032.33,1192,...,1135680,1135680-1,ATIVO,10238,20400,Stcoop,<NA>,<NA>,<NA>,<NA>
1,6174,VALÉRIO JOSÉ SEBASTIÃO,RECEITAS,17883,10354,Mensalidades/Benefícios,2024-08-12,2025-06-10,995.52,233,...,1137286,1137286-6,ATIVO,10354,17883,Stcoop,<NA>,<NA>,<NA>,<NA>
2,2593,MAICON ABATI,RECEITAS,4322,14269,Mensalidades/Benefícios,2025-04-25,2025-06-10,952.30,82,...,509,37422190000000509,ATIVO,14269,4322,Stcoop,<NA>,<NA>,<NA>,<NA>
3,8223,G B K - TRANSPORTES LTDA,RECEITAS,14562,12199,Mensalidades/Benefícios,2024-11-06,2025-06-10,2125.65,1273,...,1161436,1161436-9,ATIVO,12199,14562,Stcoop,BCK0277,93ZS3HUH0K8831150,<NA>,<NA>
4,1125,PAULO ROBERTO RABELO,RECEITAS,5598,10992,Mensalidades/Benefícios,2024-09-09,2025-06-10,975.63,1404,...,1145535,1145535-5,ATIVO,10992,5598,Stcoop,BDJ9C64,9536Y8244LR013489,<NA>,<NA>


acrescentando colunas importantes

In [7]:
df_pagamentos.loc[:, 'data_atualizacao'] = df_pagamentos['data_baixa']
df_pagamentos.loc[:, 'situacao']='PAGO'

base_vencidos.loc[:, 'data_atualizacao'] = base_vencidos['data_vencimento']
base_vencidos.loc[:, 'situacao']='INADIMPLENTE'

acrescentando colunas comuns e concatenando

In [8]:
colunas_comuns = [
    "matricula",
    "conjunto",
    "cooperativa",
    "ponteiro",
    "numero_boleto",
    "nosso_numero",
    "unidade",
    "associado",
    "data_vencimento",
    "data_baixa",
    "valor_titulo",
    "valor_baixa",
    "data_emissao",
    "dias_atrasado",
    "data_atualizacao",
    "situacao"
]

for col in colunas_comuns:
    if col not in df_pagamentos.columns:
        df_pagamentos.loc[:, col] = pd.NA
    if col not in base_vencidos.columns:
        base_vencidos.loc[:, col] = pd.NA

df_historico = pd.concat([df_pagamentos, base_vencidos], ignore_index=True)
df_historico = df_historico[colunas_comuns]


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_22224\1360921929.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos.loc[:, col] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_22224\1360921929.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_vencidos.loc[:, col] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_22224\1360921929.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version

In [9]:
df_historico['data_atualizacao'] = pd.to_datetime(df_historico['data_atualizacao'])
df_historico['data_atualizacao'] = df_historico['data_atualizacao'].dt.date


passando a base para o excel

In [10]:
save_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia"
base_vencidos.to_excel(save_path + r"\historico_faturas.xlsx", index=False, sheet_name='historico_faturas')


In [11]:
df_pagamentos.head()

,cooperativa,aplicacao,ponteiro,numero_boleto,nosso_numero,matricula,conjunto,unidade,associado,valor_baixa,data_baixa,data_vencimento,dias_atraso,data_atualizacao,situacao,valor_titulo,data_emissao,dias_atrasado
0,Stcoop,166,189381,1161651,1161651-3,20550,12202,UNIDADE VILHENA,ELI GONSALVES LIMA,1206.15,2025-03-10,2025-03-10,0,2025-03-10,PAGO,<NA>,<NA>,<NA>
21,Stcoop,166,126116,1118391,1118391-0,969,8462,UNIDADE CASCAVEL,FERNANDO MASSAHIRO KODAMA,1493.33,2025-04-15,2025-04-15,0,2025-04-15,PAGO,<NA>,<NA>,<NA>
37,Stcoop,166,184300,1184900,1184900-5,7702,11916,UNIDADE CASCAVEL,J. ANDREOLLA E CIA LTDA,1362.51,2025-03-24,2025-03-24,0,2025-03-24,PAGO,<NA>,<NA>,<NA>
64,Stcoop,166,153954,1133043,1133043-0,18542,10078,UNIDADE CASCAVEL,SIQUEIRA BATTISTI TRANSPORTES LTDA,1278.72,2025-03-05,2025-03-04,-1,2025-03-05,PAGO,<NA>,<NA>,<NA>
70,Stcoop,166,153956,1133044,1133044-7,18542,10078,UNIDADE CASCAVEL,SIQUEIRA BATTISTI TRANSPORTES LTDA,1278.72,2025-05-02,2025-05-02,0,2025-05-02,PAGO,<NA>,<NA>,<NA>


AGORA É:

OK criar a base (já tá criada) 

OK gerar a base com vencidos este MÊS

ACIMA PODE GERAR DE NOVO SÓ RODANDO ISSO AMANHÃ, ATÉ O CÓDIGO DE ATT FICAR PRONTO OU SEGUIR ABAIXO


depois gerar o código de atualização nessa ideia:

todo dia vai (pegar o excel) vai (gerar a base de inadimplencia de hoje) e vai (concatenar as duas)

(gerar uma lista com os ponteiros vencidos neste dia do mês)

(gerar a lista de baixados neste mês) e (filtrar pela lista de ponteiros vencidos)

(concatenar esses pagamentos filtrados)

(atualizar o excel)